In [1]:
import sys
sys.path.append('../')
from setting import config_read

In [2]:
import pandas as pd
from owlready2 import *
from elasticsearch import Elasticsearch
from mapping import *
from konlpy.tag import Okt

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation




In [3]:
config = config_read('../')

# Load owl file
data_path = config['owl']['path']
onto = get_ontology(data_path).load()

# Elasticsearch
server_ip = config['elasticsearch']['ip']
index_name = config['elasticsearch']['name']
es = Elasticsearch(server_ip)

In [4]:
unit_path_df = pd.read_csv('../unit_path_for_mapping.csv')

In [ ]:
okt = Okt()

query = input("Question: ") # 용감한 형제가 작곡한 노래의 장르는?
query_pos = okt.pos(query, norm=True)

query_terms = [q[0] for q in query_pos if q[1] not in ["Josa", "Punctuation"]]

In [18]:
query_patitions = query_patitioning(query_terms)
p_star, score_list = partition_scoring(es, index_name, query_patitions)
candidates_list = [onto.search(label=query) for query in p_star]
resource_combinations = list(itertools.product(*candidates_list))
resource_combinations

[(SKMO_v2.4_0223.뱅뱅뱅, SKMO_v2.4_0223.isComposedBy, foaf.Person),
 (SKMO_v2.4_0223.뱅뱅뱅_inst, SKMO_v2.4_0223.isComposedBy, foaf.Person),
 (SKMO_v2.4_0223.뱅뱅뱅_황치열, SKMO_v2.4_0223.isComposedBy, foaf.Person)]

In [19]:
onto.지드래곤.is_a

[SKMO_v2.4_0223.MusicArtist, foaf.Person]

In [20]:
conceptual_graph = []

for rc in resource_combinations:
    conceptual_arc = []
    state_list = []
    rc = list(rc)

    for r in rc:
        if r in onto.object_properties():
            state_list.append('edge') 
        elif r in onto.data_properties():
            state_list.append('edge') 
        else:
            state_list.append('vertex') 

    # 마지막이 edge일 때
    if state_list[-1] == 'edge':
        state_list.append('vertex')
        rc.append('owl.Thing')

    # vertex - vertex 일때
    index_list = []
    for i in range(len(state_list)-1):
        if state_list[i] == 'vertex' and state_list[i+1] == 'vertex':
            index_list.append(i+1)
    
    for i in index_list:
        state_list.insert(i, 'edge')
        rc.insert(i, 'Any P')
    
    for offset in range(0,len(rc)-2, 2):
        conceptual_arc.append(tuple(rc[offset:offset+3]))
    
    print(conceptual_arc)

    conceptual_graph.append(conceptual_arc)

[(SKMO_v2.4_0223.뱅뱅뱅, SKMO_v2.4_0223.isComposedBy, foaf.Person)]
[(SKMO_v2.4_0223.뱅뱅뱅_inst, SKMO_v2.4_0223.isComposedBy, foaf.Person)]
[(SKMO_v2.4_0223.뱅뱅뱅_황치열, SKMO_v2.4_0223.isComposedBy, foaf.Person)]


In [21]:
conceptual_arc = {}
df_list = []

for conceptual_arc in conceptual_graph[:1]:
    for s, p, o in conceptual_arc:
        s_cond = unit_path_df['S'] == str(s) 
        d_cond = unit_path_df['domain'] == str(s)

        p_cond = unit_path_df['P'] == str(p) 

        o_cond = unit_path_df['O'] == str(o) 
        r_cond = unit_path_df['range'] == str(o)

        df = unit_path_df.loc[(s_cond|d_cond) & p_cond & (o_cond|r_cond)]
        df_list.append(df)

In [22]:
pd.concat(df_list)

,S,P,O,domain,range,S_dpr,S_dr,W
364,SKMO_v2.4_0223.뱅뱅뱅,SKMO_v2.4_0223.isComposedBy,SKMO_v2.4_0223.지드래곤,mo.Track,foaf.Person,36,145,0.248276
382,SKMO_v2.4_0223.뱅뱅뱅,SKMO_v2.4_0223.isComposedBy,SKMO_v2.4_0223.Teddy,mo.Track,foaf.Person,36,145,0.248276
